In [ ]:
!pip install camelot-py[cv] pdfplumber pandas
#!apt-get install -y ghostscript python3-tk
!pip install camelot-py pdfplumber pandas

!pip install camelot-py
import glob
import os
import pandas as pd
import camelot
import pdfplumber

# Папки
pdf_dir = r"C:\Users\sash-\KI_multi-modal-RAG\project\pdfs"
print("Текущая рабочая директория:", os.getcwd())
output_base_dir = "parsed"
os.makedirs(output_base_dir, exist_ok=True)

# Известные компании
known_companies = ["apple", "microsoft", "google", "nvidia", "meta"]
def detect_company(file_name):
    name = file_name.lower()
    for company in known_companies:
        if company in name:
            return company.capitalize()
    return "Unknown"
pdf_files = glob.glob(os.path.join(pdf_dir, '**', '*.pdf'), recursive=True)
import os
for file_path in pdf_files:
    file_name = os.path.basename(file_path)
    
    print(f"\n📄 Обработка файла: {file_path}")
    try:
       with open(file_path, "rb") as f:
          content = f.read(1024)
          print(f"🔎 Первые 100 байт PDF: {content[:100]}")
    except Exception as e:
        print(f"❌ Ошибка чтения файла {file_path}: {e}")
        continue

company = detect_company(file_name)
output_dir = os.path.join(output_base_dir, company)
os.makedirs(output_dir, exist_ok=True)
# Очистка таблиц
def clean_financial_df(df):
    df = df.dropna(how='all')
    df = df.dropna(axis=1, how='all')
    df = df.applymap(lambda x: str(x).strip() if pd.notnull(x) else x)
    df = df.replace(r'[\$,()%B]', '', regex=True).replace(',', '', regex=True)
    df = df.apply(pd.to_numeric, errors='ignore')
    return df

# Определение компании по имени файла
def detect_company(file_name):
    name = file_name.lower()
    for company in known_companies:
        if company in name:
            return company.capitalize()
    return "Unknown"

# Обработка всех PDF-файлов
pdf_files = glob.glob(os.path.join(pdf_dir, '**', '*.pdf'), recursive=True)

for file_path in pdf_files:
    file_name = os.path.basename(file_path)
    base_name = os.path.splitext(file_name)[0]
    company = detect_company(file_name)

    output_dir = os.path.join(output_base_dir, company)
    os.makedirs(output_dir, exist_ok=True)

    print(f"\n📄 Обработка: {file_name} →  {company}")

    print(f"🔍 Проверка чтения файла: {file_path}")
    with open(file_path, "rb") as f:
       content = f.read(1024)
       print(f"🔎 Первые 100 байт PDF: {content[:100]}")

    # 🐫 Camelot (оба flavor)
    for flavor in ['lattice', 'stream']:
        try:
            tables = camelot.read_pdf(file_path, pages='all', flavor=flavor)  #вызывается Ghostscript, и
            print(f"📊 Camelot ({flavor}) нашел {tables.n} таблиц")  # ← отладочный вывод

            if tables.n > 0:
                for i, table in enumerate(tables):
                    df = clean_financial_df(table.df)   #Vorreinigung
                    
                    print(df.head())  # ← просмотр таблицы
                    csv_name = f"{base_name}_camelot_{flavor}_table{i+1}.csv"
                    csv_path = os.path.join(output_dir, csv_name)
                    df.to_csv(csv_path, index=False)
                    print(f" ✅ Camelot ({flavor}) → {csv_path}")
        except Exception as e:
            print(f" ⚠️ Ошибка Camelot ({flavor}): {e}")


     # 🪶 PDFPlumber (всегда)
    try:
        with pdfplumber.open(file_path) as pdf:
            for page_num, page in enumerate(pdf.pages):
                tables = page.extract_tables()
                print(f"📄 PDFPlumber: finden {len(tables)} tabelle site {page_num+1}")
                for j, table in enumerate(tables):
                    if table:  # иногда возвращается пустая таблица
                        df = pd.DataFrame(table)
                        df = clean_financial_df(df)
                        print(df.head())  # ← просмотр таблицы
                        csv_name = f"{base_name}_plumber_page{page_num+1}_table{j+1}.csv"
                        csv_path = os.path.join(output_dir, csv_name)
                        df.to_csv(csv_path, index=False)
                        print(f" ✅ PDFPlumber → {csv_path}")
    except Exception as e:
        print(f" ⚠️ Fehler PDFPlumber: {e}")

import glob
csv_files = glob.glob("parsed/**/*.csv", recursive=True)

print("\n📁 Überprüfung erstellt CSV:")
csv_files = glob.glob("parsed/**/*.csv", recursive=True)
if not csv_files:
    print("❌ CSV-Dateien nicht gefunden!")
else:
    for f in csv_files:
        print(f"✅ Найден файл: {f}")
print("Aktueller Arbeitsordner:", os.getcwd())
print("Gefundene Tabellen::", tables.n)
if tables.n > 0:
    tables[0].to_csv("parsed/test_table.csv")
    print("✅ Saved as test_table.csv")
else:
    print("⚠️ No tables found")
import os

tables = camelot.read_pdf(file, pages="1", flavor="stream")

#parset
#pdfs
#Proekt7_pdf_csv

#print(f"Найдено таблиц: {tables.n}")  # для Camelot
#print(table)  # для PDFPlumber, чтобы увидеть содержимое
#print(f"📊 Camelot ({flavor}) нашел {tables.n} таблиц")
#print(df.head())
-